# Tarea 3 Cadenas de Markov


Autores: 
- Daniel Alejandro García Hernández
- David Camilo Cortes Salazar

En este notebook se encuentra una implementación de algunos resultados vistos en clase para el Gibbs Sampler aplicado a Hard-core y q-colorings. 

Las librerías necesarias para ejectuar el código son:

In [ ]:
import numpy as np
import random
from pprint import pprint

# Modelo de Ising y Algoritmo de Propp - Wilson

### Muestras del modelo de Ising con inverso de temperatura usando MCMC

Iniciamos creando la grilla del módelo. Para esto, creamos una matriz 2x2 apoyandonos en la libreria Numpy y la declaramos de tamaño $kxk$. las dimensiones del grafo. $10\leq k \leq 20$.

Tambien creamos una lista con los beetas que vamos a evaluar y otra con los diferentes numeros de pasos con los que ejecturemos el algoritmo.

In [14]:
k = 10
beta_list = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
steps_list = [10**3, 10**4, 10**5]

Ahora recordemos como se calculaba la energia del sistema dada una configuracion. Esto esta dado por la funcion $H(\eta)$ descrita acontinuacion:

$$H(\eta)=-\sum_{x\thicksim y}\eta_{x}\eta_{y}$$

Donde $\eta$ es la configuracion actual del sistema y $\eta_{x}$, $\eta_{y}$ son la posicion de dos nodos de $\eta$ tal que $x$ y $y$ estan conectados.

Con esto en mente creamos una funcion que calcula la energia del sistema dada una configuracion $\eta$ multiplicando todos los nodos conectados.

In [29]:
def calcular_energia(G, k):
    # Calcula la energia de la configuracion dada
    energia = 0
    for i in range(k):
        for j in range(k):
            if i != k-1:
                energia += G[i+1, j]
            if j != k-1:
                energia += G[i, j+1]
    return -1 * energia

Ahora usemos un gibbs sampler para implementar el metodo de montecarlo en nuestro problema. Este funcionara de la siguiente manera:

- Seleccione un vertice al azar de la configurtacion actual $\eta$.
- Cambie el valor del vertice escogido (Si el vertice es 1, cambie por -1 y viceversa). Esto para obtener una nueva configuracion $\hat{\eta}$ del sistema.
- Calcule la energia de las configuraciones $\eta$ y $\hat{\eta}$.
- Calcule la probabilidad de cambio de $\eta$ y $\hat{\eta}$ utilizando la razon entre sus distribuciones de probabilidad: 

$$ \Delta \pi = \frac{\pi_{\beta}(\hat{\eta})}{\pi_{\beta}(\eta)} = \exp^{2\beta (H(\hat{\eta})-H(\eta))} $$

- Genere un numero aleatorio uniforme $p$ entre 0 y 1
- Si $p < \Delta \pi$, acepta la configuracion $\hat{\eta}$. Si no, mantenemos con la configuracion $\eta$.
- Repetimos el algoritmo hasta completar las iteraciones propuestas.

In [42]:
def gibbs_sampler_mcmc(G, beta, iteraciones, k):
    for _ in range(iteraciones):
        
        # Selecciona spin aleatorio
        x = random.randint(0,k-1)
        y = random.randint(0,k-1)

        # Siguiente configuracion segun el spin escogido
        G_sig = G.copy()
        G_sig[x, y] *= -1

        # Calcula la energia de la configuracion actual y siguiente segun el spin dado
        energia_actual = calcular_energia(G, k) # Energia con configuracion actual
        energia_sig = calcular_energia(G_sig, k) # Energia con configuracion propuesta

        # Calcula la probabilidad de cambio de la nueva configuracion
        if beta != 0:
            prob_cambio = np.exp(2 * beta * (energia_sig-energia_actual))
        else:
            prob_cambio = 1
        
        # Condicion de Cambio
        if np.random.rand() < prob_cambio: # Aplica Monte Carlo
            G[x, y] *= -1 # Cambia el spin
    
    return G

Ahora solo nos queda probar el algoritmo con los valores de beta y pasos del gibbs sampler que definimos al inicio.

In [43]:
configuraciones = {}

# Loop principal
for beta in beta_list:
    configuraciones[beta] = []
    for step in steps_list:
        G = np.random.choice([-1, 1], size=(k, k))  # Configuración inicial aleatoria
        configuraciones[beta].append(gibbs_sampler_mcmc(G, beta, step, k).copy())  # Guarda la configuracion final

pprint(configuraciones)

{0: [array([[-1,  1,  1, -1, -1,  1, -1, -1,  1,  1],
       [ 1,  1, -1, -1,  1,  1,  1, -1,  1, -1],
       [ 1,  1,  1,  1,  1,  1, -1,  1,  1,  1],
       [-1,  1, -1, -1, -1,  1, -1,  1,  1,  1],
       [-1, -1,  1, -1,  1, -1,  1,  1, -1, -1],
       [-1,  1,  1, -1, -1,  1,  1,  1, -1,  1],
       [ 1,  1,  1,  1,  1,  1, -1, -1, -1, -1],
       [ 1, -1, -1,  1,  1, -1,  1,  1, -1,  1],
       [-1, -1,  1,  1, -1,  1,  1, -1, -1,  1],
       [-1, -1, -1,  1,  1,  1, -1,  1,  1,  1]]),
     array([[ 1, -1, -1,  1, -1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1,  1, -1, -1,  1, -1],
       [ 1,  1, -1,  1, -1,  1,  1, -1,  1, -1],
       [-1,  1, -1,  1,  1,  1, -1,  1,  1, -1],
       [-1,  1,  1,  1, -1,  1, -1,  1, -1,  1],
       [-1,  1,  1,  1, -1, -1,  1, -1, -1, -1],
       [ 1, -1, -1,  1,  1,  1,  1, -1, -1,  1],
       [ 1,  1,  1, -1, -1, -1, -1,  1, -1, -1],
       [ 1, -1, -1,  1, -1,  1, -1,  1,  1, -1],
       [-1, -1,  1,  1, -1, -1,  1, -1,  1,  1]]),
     a

Como ultimo paso, generamos 100 muestras aproximadas

In [44]:
# 100 por cada uno de los pasos y epsilons?